In [ ]:
# Importing the libraries needed
import torch
import random
import numpy as np
import transformers

from torch import nn

SEED = 42

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [ ]:
from torch import cuda

device = 'cuda' if cuda.is_available() else 'cpu'

In [ ]:
import pandas as pd

train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')
submit = pd.read_csv('./sample_submission.csv')

test_list = test["Utterance"].values.tolist()

In [ ]:
train.head()

In [ ]:
train['Target'].unique()

In [ ]:
test_list[0]

**Making a model inference as a function**

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

def sentiment_classification(modelname):
    result = list()
    final_labels = list()

    model = modelname

    tokenizer = AutoTokenizer.from_pretrained(model)    
    
    model = AutoModelForSequenceClassification.from_pretrained(model)

    classification = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, device=0)

    for i in range(len(test_list)):
        print(classification(test_list[i]))
        result.append(classification(test_list[i]))

    for i in range(len(result)):
        for j in range(len(result[i])):
            final_labels.append([result[i][j]['label'], result[i][j]['score']])
    
    return final_labels

**base model inference**

EmoBERTa paper : https://arxiv.org/abs/2108.12009  
(Base) HuggingFace link : https://huggingface.co/tae898/emoberta-base?text=I+like+you.+I+love+you

In [ ]:
inference01 = sentiment_classification("tae898/emoberta-base")

In [ ]:
inference01[:5]

**large model inference**

(Large) HuggingFace link : https://huggingface.co/tae898/emoberta-large?text=I+like+you.+I+love+you

In [ ]:
inference02 = sentiment_classification("tae898/emoberta-large")

In [ ]:
inference02[:5]

In [ ]:
final_concatenation = list()

for i in range(len(inference01)):
    if max(float(inference01[i][1]), float(inference02[i][1])) == float(inference01[i][1]):
        final_concatenation.append(inference01[i][0])
    elif max(float(inference01[i][1]), float(inference02[i][1])) == float(inference02[i][1]):
        final_concatenation.append(inference02[i][0])
    elif float(inference01[i][1]) == float(inference02[i][1]) :
        final_concatenation.append(inference01[i][0])

In [ ]:
final_concatenation[0]

In [ ]:
submit['Target'] = final_concatenation
submit.head()

In [ ]:
submit.to_csv('./HuggingFace_pipeline_concatenation.csv', index=False)